In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

1- importer les librairies

In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import xgboost as xgb
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
import warnings
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import ShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
warnings.filterwarnings('ignore')

%matplotlib inline

2 - importer le dataset

In [ ]:
train = pd.read_csv ('../input/train.csv')
test = pd.read_csv ('../input/test.csv')
sample_submission = pd.read_csv ('../input/sample_submission.csv')

In [ ]:
train.shape,test.shape

3- Etude du dataset

In [ ]:
train.head(20)

In [ ]:
test.head(20)

In [ ]:
train.info()

In [ ]:
test.info()

Check ou and Detecting the missing Values

In [ ]:
train.isnull().sum()

In [ ]:
train.dropna(inplace=True)
train.isnull().sum()

In [ ]:
train.trip_duration.min()

In [ ]:
train.trip_duration.max()

see the categorical values

cleaning the data

In [ ]:
#gestion des outliers
plt.subplots(figsize=(17,7))
plt.title("outliers repartition")
train.boxplot()

In [ ]:
train.loc[train.trip_duration<5000,"trip_duration"].hist(bins=150
                                                        )

Dat visualisation et analyse

In [ ]:
train['log_trip_duration'] = np.log(train['trip_duration'])
plt.hist(train['log_trip_duration'].values, bins=50)
plt.xlabel('log(trip_duration)')
plt.ylabel('number of train records')
plt.show()

In [ ]:
import warnings
warnings.filterwarnings("ignore")
plot_vendor = train.groupby('vendor_id')['trip_duration'].mean()
plt.subplots(1,1,figsize=(17,10))
plt.ylim(ymin=800)
plt.ylim(ymax=840)
sns.barplot(plot_vendor.index,plot_vendor.values)
plt.title('Time per Vendor')
plt.legend(loc=0)
plt.ylabel('Time in Seconds')

In [ ]:
snwflag = train.groupby('store_and_fwd_flag')['trip_duration'].mean()
plt.subplots(1,1,figsize=(18,10))
plt.ylim(ymin=0)
plt.ylim(ymax=1100)
plt.title('Time per store_and_fwd_flag')
plt.legend(loc=0)
plt.ylabel('Time in Seconds')
sns.barplot(snwflag.index,snwflag.values)

In [ ]:
train['log_trip_duration'] = np.log(train['trip_duration'].values + 1)
plt.hist(train['log_trip_duration'].values, bins=100)
plt.xlabel('log(trip_duration)')
plt.ylabel('number of train records')
plt.show()

In [ ]:
pc = train.groupby('passenger_count')['trip_duration'].mean()

plt.subplots(1,1,figsize=(17,10))
plt.ylim(ymin=0)
plt.ylim(ymax=1100)
plt.title('Time per store_and_fwd_flag')
plt.legend(loc=0)
plt.ylabel('Time in Seconds')
sns.barplot(pc.index,pc.values)


In [ ]:
import matplotlib.pyplot as plt 
from matplotlib import animation
from matplotlib import cm
import base64
import io
longitude = list(train.pickup_longitude) + list(train.dropoff_longitude)
latitude = list(train.pickup_latitude) + list(train.dropoff_latitude)
plt.figure(figsize = (10, 10))
plt.plot(longitude,latitude,'.', alpha = 1, markersize = 25)
plt.show()

In [ ]:
train.groupby('passenger_count').size()

In [ ]:
test.groupby('passenger_count').size()

In [ ]:
train.loc[train.trip_duration<5000,"trip_duration"].hist(bins=50
                                                              )

selection exctraction création 

In [ ]:
col_diff = list(set(train.columns).difference(set(test.columns)))
col_diff

In [ ]:
train['dist'] = abs((train['pickup_latitude']-train['dropoff_latitude'])
                        + (train['pickup_longitude']-train['dropoff_longitude']))
test['dist'] = abs((test['pickup_latitude']-test['dropoff_latitude'])
                        + (test['pickup_longitude']-test['dropoff_longitude']))

In [ ]:
train.columns

In [ ]:
train.pick_datetime = pd.to_datetime(train.pickup_datetime)
test.pick_datetime = pd.to_datetime(test.pickup_datetime)

In [ ]:
train['store_and_fwd_flag']

In [ ]:
col_drop = ["id"]
y = train["log_trip_duration"]
train.drop(["id","trip_duration","dropoff_datetime"], axis=1, inplace=True)
X = train
X.shape, y.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, random_state=42)
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

In [ ]:
rf1 = RandomForestRegressor( n_estimators=100, min_samples_leaf=5 )

In [ ]:
def featuresSelection(df_in):
    VARS_NUM = ['vendor_id', 'pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude', 'distances', 'pickup_Year', 'pickup_Month', 'pickup_Hour', 'pickup_Weekday' ] # 'passenger_count',
    vars_num = VARS_NUM

   
    X=df_in.loc[:, vars_num]


    return X

In [ ]:
from sklearn.neighbors import KNeighborsClassifier